## 回归的线性模型

- 使用pytorch求解线性回归问题（超定方程的最小二乘解）
    - 使用pytorch.gels（对应的是正规方程的解析闭式解）
    - 使用pytorc.nn.Linear，相当于对gels的封装，例如不用为设计矩阵加1等
    - 使用优化器. 当超定方程过大，无法一次加载，可使用基于梯度下降的优化算法进行迭代求数值解
   

- 损失函数的分类
    - L1
    - L2
    - SmoothL1
 

- 数据的标准化
    - 标准化的作用

In [1]:
import numpy as np
import torch

##1.1-多元线性回归
X = torch.tensor([[1,1,1,],[2,3,1],[3,5,1],[4,2,1],[5,4,1]],dtype=torch.float)
y = torch.tensor([-10,12,14,16,18],dtype=torch.float)
w,_=torch.gels(y,X)##这里的顺序不能错
w[:3]

tensor([[  4.6667],
        [  2.6667],
        [-12.0000]])

In [2]:
##1.2-多变量线性回归
Y=torch.tensor([[-10,3],[12,14],[14,12],[16,16],[18,16]],dtype=torch.float)
W,_=torch.gels(Y,X)
W[:3,:]

tensor([[  4.6667,   2.5333],
        [  2.6667,   0.5333],
        [-12.0000,   3.0000]])

In [4]:
##2.损失函数
E=torch.nn.MSELoss()
y=torch.arange(5,requires_grad=True,dtype=torch.float)
t=torch.ones(5)

##2.1-得到均方误差损失
loss=E(y,t)##注意这里的顺序不能错，t属于标签数据，y是预测数据，要对y相关的变量求梯度
loss

tensor(3., grad_fn=<MseLossBackward>)

In [5]:
torch.mean((y-t)**2)

tensor(3., grad_fn=<MeanBackward1>)

In [6]:
##2.2-L1 Loss
E_L1=torch.nn.L1Loss()
loss2=E_L1(y,t)
loss2

tensor(1.4000, grad_fn=<L1LossBackward>)

In [7]:
##2.3-SmoothL1 Loss
E_SL1=torch.nn.SmoothL1Loss()
loss3=E_SL1(y,t)
loss3

tensor(1., grad_fn=<SmoothL1LossBackward>)

In [9]:
##3.使用优化器求解回归问题的解
X

tensor([[1., 1., 1.],
        [2., 3., 1.],
        [3., 5., 1.],
        [4., 2., 1.],
        [5., 4., 1.]])

In [11]:
y = torch.tensor([-10,12,14,16,18],dtype=torch.float)
y

tensor([-10.,  12.,  14.,  16.,  18.])

In [14]:
w=torch.zeros(3,requires_grad=True,dtype=torch.float)
E=torch.nn.MSELoss()
opt=torch.optim.Adam([w])
loss=E(torch.mv(X,w),y)

for i in range(30000):
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    loss=E(torch.mv(X,w),y)
    
    if i%1000 == 0:
        print('第{}次：loss={},w={},grad_w={}'.format(i,np.round(loss.tolist(),3),  
                                                   np.round(w.tolist(),3),
                                                   np.round(w.grad.tolist(),3)))

第0次：loss=203.816,w=[0.001 0.001 0.001],grad_w=[-84. -80. -20.]
第1000次：loss=86.777,w=[0.864 0.855 0.822],grad_w=[-43.001 -39.018  -8.054]
第2000次：loss=53.616,w=[1.444 1.395 1.19 ],grad_w=[-17.205 -13.303  -0.59 ]
第3000次：loss=47.502,w=[1.784 1.634 0.957],grad_w=[-6.32  -2.619  2.425]
第4000次：loss=44.285,w=[2.032 1.702 0.255],grad_w=[-3.725 -0.383  2.915]
第5000次：loss=40.87,w=[ 2.305  1.713 -0.619],grad_w=[-2.743  0.072  2.87 ]
第6000次：loss=37.534,w=[ 2.62   1.685 -1.545],grad_w=[-1.932  0.2    2.74 ]
第7000次：loss=34.523,w=[ 2.951  1.636 -2.48 ],grad_w=[-1.233  0.137  2.564]
第8000次：loss=31.943,w=[ 3.26   1.603 -3.407],grad_w=[-0.672  0.015  2.362]
第9000次：loss=29.763,w=[ 3.504  1.627 -4.322],grad_w=[-0.307 -0.061  2.141]
第10000次：loss=27.898,w=[ 3.679  1.713 -5.221],grad_w=[-0.137 -0.068  1.908]
第11000次：loss=26.302,w=[ 3.816  1.83  -6.103],grad_w=[-0.073 -0.046  1.668]
第12000次：loss=24.956,w=[ 3.943  1.95  -6.964],grad_w=[-0.043 -0.028  1.429]
第13000次：loss=23.85,w=[ 4.064  2.069 -7.803],grad_w=[-

In [19]:
##4.使用torch.nn.Linear求解线性回归问题
X=X[:,:2]
X

tensor([[1., 1.],
        [2., 3.],
        [3., 5.],
        [4., 2.],
        [5., 4.]])

In [28]:
y=y.reshape(-1,1)
y.shape

torch.Size([5, 1])

In [33]:
##生成一个全连接层：2个输入，1个输出
##或者说是该层有1个神经元，这个神经元有2个突触

fc=torch.nn.Linear(2,1)##权值包含在这个fc中
E=torch.nn.MSELoss()
opt=torch.optim.Adam(fc.parameters())

pred=fc(X)
loss=E(fc(X),y)

w,b=fc.parameters()

for i in range(30000):
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    loss=E(fc(X),y)
    
    
    if i%1000 ==0:
        print('第{}次：loss={},w={},grad_w={},b={},grad_b={}'.format(i,np.round( loss.tolist() ,3)
                                                                  ,w,w.grad.tolist(),b,b.grad.tolist()))

第0次：loss=162.092,w=Parameter containing:
tensor([[0.0479, 0.5082]], requires_grad=True),grad_w=[[-72.6204833984375, -67.69975280761719]],b=Parameter containing:
tensor([0.0349], requires_grad=True),grad_b=[-16.607486724853516]
第1000次：loss=69.947,w=Parameter containing:
tensor([[0.8876, 1.3322]], requires_grad=True),grad_w=[[-32.99868392944336, -28.10956573486328]],b=Parameter containing:
tensor([0.8101], requires_grad=True),grad_b=[-5.069876670837402]
第2000次：loss=50.266,w=Parameter containing:
tensor([[1.4065, 1.7768]], requires_grad=True),grad_w=[[-11.610899925231934, -6.8700408935546875]],b=Parameter containing:
tensor([0.9874], requires_grad=True),grad_b=[1.0705972909927368]
第3000次：loss=46.204,w=Parameter containing:
tensor([[1.7154, 1.9131]], requires_grad=True),grad_w=[[-5.117435932159424, -0.7214572429656982]],b=Parameter containing:
tensor([0.4808], requires_grad=True),grad_b=[2.732027053833008]
第4000次：loss=42.606,w=Parameter containing:
tensor([[2.0039, 1.9214]], requires_grad=

In [53]:
v=torch.arange(1,6,3)
v.dtype

torch.int64

In [52]:
i=v.double()
i.dtype

torch.float64

In [112]:
x = torch.Tensor([.1])


In [60]:
w=torch.tensor([2,2,2],dtype=torch.float,requires_grad=True)
y=x*w
z=y.sum()
z.backward()

In [61]:
x.grad

tensor([2., 2., 2.])

In [62]:
w.grad

tensor([1., 2., 3.])

In [67]:
type(z)

torch.Tensor

In [68]:
type(x)

torch.Tensor

In [139]:
from torch.autograd import Variable as Var
xv=Var(x,requires_grad=True)
wv=Var(w)
yv=xv*wv
zv=yv.sum()
zv.backward(retain_graph=True)
xv.grad

tensor([6.])

In [140]:
xv.grad

tensor([6.])

In [74]:
type(zv)

torch.Tensor

In [75]:
zv.grad_fn

In [76]:
zv.grad

In [113]:
yv.grad_fn

In [119]:
yv.grad_fn.next_functions

((<AccumulateGrad at 0x2afbdbd94a8>, 0), (None, 0))

In [79]:
zv.data

tensor(12.)

In [120]:
zv.grad_fn.next_functions

((<ThMulBackward at 0x2afbdbce978>, 0),)

In [82]:
type(zv.grad)

NoneType

In [122]:
xv.requires_grad

True

In [84]:
type(xv)

torch.Tensor

In [89]:
i=torch.tensor([1,2,3],dtype=torch.int)
i

tensor([1, 2, 3], dtype=torch.int32)

In [90]:
i.type()

'torch.IntTensor'

In [107]:
type(torch.FloatTensor)==type(torch.FloatTensor)

True

In [143]:
zv.backward()

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [142]:
xv.grad

tensor([12.])

In [129]:
yv.grad_fn

In [149]:
type(xv.size())

torch.Size

True